In [ ]:
import pypsa
import pandas as pd
import matplotlib.pyplot as plt

Read network

In [ ]:
n=pypsa.Network("../../results/nz_hydro_eia/postnetworks/elec_s_10_ec_lc1.0_Co2L_3H_2030_0.071_AB_20export.nc")

In [ ]:
n.generators[n.generators.carrier=="onwind"].p_nom_max.sum() /1e3

In [ ]:
n.generators[n.generators.carrier=="solar"].p_nom_max.sum() /1e3

In [ ]:
stats = n.statistics()

In [ ]:
(n.statistics.optimal_capacity()["Generator"]/1e3).round(1) # Capacity in GW

### Dispatch

In [ ]:
(n.statistics.dispatch()[n.statistics.dispatch()>1e3]/ 1e6).round(1)  # Energy in TWh

In [ ]:
(n.statistics.dispatch()[n.statistics.dispatch()>1e3]/ 1e6).round(1)  # Energy in TWh

### Hydro analysis

In [ ]:
n.storage_units

In [ ]:
n.storage_units.iloc[0][0:20]

In [ ]:
n.storage_units_t.p.iloc[0:200].plot() #sum()

In [ ]:
n.storage_units.iloc[0]

In [ ]:
n.storage_units.inflow

In [ ]:
n.statistics.capex()[n.statistics.capex()>1e6] / 1e6

Hydro analysis PyPSA-Earth

In [ ]:
n_earth = pypsa.Network("../../results/pypsa-earth/nz_hydro_eia/networks/elec_s.nc")

In [ ]:
n_earth.storage_units.iloc[0][0:20]

In [ ]:
n_earth.storage_units_t.inflow